In [212]:
from pyspark import SparkContext

# # Loading data from csv to RDD

In [ ]:
bnk1 = sc.textFile("/home/anil/Anil_Created_Docs/01.02.BID DATA/PySPark/anil_trading/data/6440633_01042019_06032020.csv")
bnk1.take(5)

In [ ]:
bnk3 = bnk1.zipWithIndex().filter(lambda x : x[1] > 0).map(lambda x : x[0])
bnk3.take(5)

In [ ]:
bnk2 = bnk1.map(lambda x : x.split(","))
bnk2.take(5)

# Loading data from csv to DataFrame

In [ ]:
data_schema = [StructField('Date', StringType(), True)
               , StructField('Narration', StringType(), True)
               , StructField('Chq_Ref_No', StringType(), True)
               , StructField('Value_Date', StringType(), True)
               , StructField('Withdrawl_Amt', IntegerType(), True)
               , StructField('Deposit_Amt', IntegerType(), True)
               ,  StructField('Closing_Bal', IntegerType(), True)
              ,  StructField('Closing_Bal1', IntegerType(), True)]
final_struc = StructType(fields=data_schema)
df = spark.read.csv('/home/anil/Anil_Created_Docs/01.02.BID DATA/PySPark/anil_trading/data/6440633_01042019_06032020.csv', sep=",",header=True 
                    ,inferSchema=True
                    #, schema=final_struc
                   )

In [ ]:
df.take(1)

In [ ]:
df.select("Date","Closing Balance").collect()

In [ ]:
df.selectExpr()
#https://stackoverflow.com/questions/34803855/pyspark-dataframe-udf-on-text-column?rq=1

In [ ]:
df.show()

In [ ]:
df.select("Narration").collect()

In [ ]:
pattern = ["DEPOSITORY CHARGES","NET PI TO HSL","NET PO FROM HSL","IRCTC","APSRTC","CAPITALISED","ATW-","SALARY CREDIT"]

for x in range(len(pattern)):
    print(pattern[x])


In [ ]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
import re


def findTxnCategory(value):
    pattern_list = ["DEPOSITORY CHARGES","NET PI TO HSL","NET PO FROM HSL","IRCTC","APSRTC","CAPITALISED"
                    ,"ATW-","SALARY CREDIT","DELHI PUBLIC SCHOOL","SBI CARDS"]
    for x in range(len(pattern_list)):
        pattern = re.compile(pattern_list[x])     
        if pattern.search(value):
            #print("Yes")
            return pattern_list[x]
        #else:
        #    print("No")
        #    return "No"
    return "OTHERS"

func_FindTxnCategory = F.udf(findTxnCategory,StringType())

In [ ]:
print(findTxnCategory("MY IRCTC MONEY"))

In [ ]:
df1 = df.withColumn("TXN_CATEGORY",func_FindTxnCategory("Narration"))

In [ ]:
df1.select("TXN_CATEGORY").collect()

In [ ]:
df2 = df1.groupBy("TXN_CATEGORY").count().collect()

In [ ]:
df2

In [ ]:
df1.columns

In [ ]:
df1.select()

df1.printSchema()

In [ ]:
df2 = (df1.withColumnRenamed('Deposit Amt.', 'Deposit_Amt')
       .withColumnRenamed('Withdrawal Amt.', 'Withdrawal_Amt')
       .withColumnRenamed('Closing Balance', 'Closing_Bal')
      )
df2.printSchema()

In [ ]:
df2.groupBy("TXN_CATEGORY").sum().show()

In [ ]:
df2.groupBy("TXN_CATEGORY").agg({"Withdrawal_Amt":"sum","Deposit_Amt":"sum","Narration":"count"}).filter(df2["TXN_CATEGORY"] == 'OTHERS').show()